In [55]:
# Tried Step 1 with this file earlier but got only 1000 posts, so moved to another API

# pip install praw scikit-learn pandas
import requests
import praw
import json
import pandas as pd
import datetime as dt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier

In [56]:
client_auth = requests.auth.HTTPBasicAuth('do8wnlq4TqLxbw', 'BTs2v7p53_Gbke6HeBQ6ysaDOTg')

post_data = {"grant_type": "password", "username": "ndjad", "password": "fjanfj@"}

In [57]:
headers = {"User-Agent": "utsavgoel"}
response = requests.post("https://www.reddit.com/api/v1/access_token", auth=client_auth, data=post_data, headers=headers)
response = response.json()

In [58]:
headers = {"Authorization": "bearer " + response['access_token'], "User-Agent": "utsavgoel"}
response = requests.get("https://oauth.reddit.com/api/v1/me", headers=headers)

In [59]:
reddit = praw.Reddit(client_id='do8wnlq4TqLxbw',
                     client_secret='BTs2v7p53_Gbke6HeBQ6ysaDOTg',
                     user_agent='utsavgoel',
                     username='utsavgoel',
                     password='utsav11@')

In [117]:
topics_dict = { "title":[], 
                "score":[], 
                "id":[], "url":[],  
                "comms_num": [], 
                "created": [], 
                "body":[],
                "flair":[]}

for submission in reddit.subreddit('india').hot(limit=1000):
    if submission.link_flair_text:
        #print(submission.link_flair_text)
        topics_dict["title"].append(submission.title)
        topics_dict["score"].append(submission.score)
        topics_dict["id"].append(submission.id)
        topics_dict["url"].append(submission.url)
        topics_dict["comms_num"].append(submission.num_comments)
        topics_dict["created"].append(submission.created)
        topics_dict["body"].append(submission.selftext)
        topics_dict["flair"].append(submission.link_flair_text)
    
train_data = pd.DataFrame(topics_dict)

for submission in reddit.subreddit('india').top(limit=1000):
    if(submission.link_flair_text):
        #print(submission.link_flair_text)
        topics_dict["title"].append(submission.title)
        topics_dict["score"].append(submission.score)
        topics_dict["id"].append(submission.id)
        topics_dict["url"].append(submission.url)
        topics_dict["comms_num"].append(submission.num_comments)
        topics_dict["created"].append(submission.created)
        topics_dict["body"].append(submission.selftext)
        topics_dict["flair"].append(submission.link_flair_text)
    
test_data = pd.DataFrame(topics_dict)

In [126]:
def get_date(created):
    return dt.datetime.fromtimestamp(int(created)).strftime('%Y-%m-%d')
    #return dt.datetime.date(created)

#convert unix time to normal date
_timestamp = train_data["created"].apply(get_date)
train_data = train_data.assign(created = _timestamp)
train_data.to_csv('reddit.csv', index=False) 

In [127]:
text_clf = Pipeline([('vect', CountVectorizer(token_pattern=r'\b[^\d\W]+\b', stop_words='english')),
                    ('tfidf', TfidfTransformer()),
                    ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                        alpha=1e-3, random_state=42)),
])
text_clf = text_clf.fit(train_data['title'], train_data['flair'])
print(text_clf)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words='english', strip_accents=None,
                                 token_pattern='\\b[^\\d\\W]+\\b',
                                 tokenizer=None, vocabulary...
                ('clf-svm',
                 SGDClassifier(alpha=0.001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                  